In [ ]:
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Electric Energy
- calculation of the electric power consumption for a certain building


## Preprocess data

In [ ]:
# load csv
b = pd.read_csv('../data/preprocessed/Building_4.csv').astype({'datetime': 'datetime64'}) # keep index
# convert datetime column to pd datetime format
b.datetime = pd.to_datetime(b.datetime)
# Extract additional features from timestamp column
b = b.assign(
    timestamp = b['datetime'], # timestamp column is useful for seaborn lineplots
    #date = lambda x: x['timestamp'].dt.date,
    year = lambda x: x['timestamp'].dt.year,
    month = lambda x: x['timestamp'].dt.month,
    day = lambda x: x['timestamp'].dt.day,
    dayofweek = lambda x: x['timestamp'].dt.dayofweek,
    dayofyear = lambda x: x['timestamp'].dt.dayofyear,
    hour = lambda x: x['timestamp'].dt.hour,
)
b.set_index('datetime',inplace=True) # set the datetime column as index column
b.head()

In [ ]:
# create data snipped in a certain time range
b_snip = b.loc[pd.date_range('2011-07-07 00:00', '2011-07-14 23:00', freq='H')]

In [ ]:
# get an overview by plotting the data
plt.figure(figsize=(20,5))
sns.lineplot(data=b_snip,x='datetime',y='equipment_electric_power_kWh')
sns.lineplot(data=b_snip,x='datetime',y='dhw_heating_kWh')
sns.lineplot(data=b_snip,x='datetime',y='cooling_load_kWh')
sns.lineplot(data=b_snip,x='datetime',y='solar_generation_kW')
plt.grid('major')

## Calculation of electric power consumption of the heat pump and the electric heater for DHW

### Heat pump coefficient of performance (COP)

In [ ]:
# calculate heat pump power consumption
# parameter set up
eta_hpc = 0.15; # technical efficiency coefficient 0.2 ... 0.3 typically
temp_target_c = 5; # target temperature for hp cooling, typically 7 ... 10 °C, this equals approx. the temperature of the chilled water storage
T_th = 273.15; # thermodynamic temperature in Kelvin [K] corresponding to 0 °C 
# adding a COP column to the data frame
b_snip['COP_c'] = eta_hpc * (temp_target_c + T_th)/ ((b_snip['outdoor_temp'] + T_th) - (temp_target_c + T_th)) 

In [ ]:
# COP plot of hp cooling incl. mean COP value (horizontal line)
plt.figure(figsize=(20,5))
sns.lineplot(data=b_snip,x='datetime',y='COP_c')
#sns.lineplot(data=b4_snip,x='datetime',y='outdoor_temp')
plt.axhline(b_snip['COP_c'].mean())
#plt.axhline(0)
#plt.ylim(-10,25)
plt.title('COP of hp cooling over time');

- theoretical max COP for cooling is approx. 7.8, typical values are 2 ... 4; so the values seem realistic, but tend to be a little bit high
- this hp seems to perform well, the technical efficiency coeff. is already reduced to 0.15 to not have an unrealistic well performing device


### Details on COP

- how is the COP changing over outdoor temperature?
- what is the limit of the approximated equation for COP calculation?

In [ ]:
# cop over outdoortemp
# create temperature list
from numpy import linspace

temp = list(linspace(0,35,350)) # create list of temperature between 0 and 35 °C
df_cop = pd.DataFrame(temp,columns=['temp']) # create dataframe from temperature list
df_cop['cop'] = eta_hpc * (temp_target_c + T_th)/ ((df_cop['temp'] + T_th) - (temp_target_c + T_th)) # add a COP column
print('The constant target temperature for cooling is: ',temp_target_c)
print('\nMin outdoor temp and max values of realistic cop: \n', df_cop.loc[(df_cop['cop']<7.8)&(df_cop['cop']>0)].head(1)) # realistic is a cop between 0 and 7.8
plt.figure(figsize=(20,5))
sns.lineplot(data=df_cop,x='temp',y='cop')
plt.title('COP cooling over outdoor temperature')


- if the difference between outdoor temp and target temp for cooling gets lower than 5 K, the calculated COP increases to unrealistic values, thus minimum difference between outdoor and target temperature should be delta_Tc_min = approx. 5
- in summer, the proposed target temp of 7 ... 10 °C works fine, in winter a lower target has to be chosen (approx. 3 ... 5 °C). The following graph - COP over outdoor_temp - shows how the cop changes depending on outdoor temperature with a constant target temperature). Only values for temp > (target_temp + delta_Tc_min) are realistic!
- in general: the lower the target temp (temperature of the chilled water storage), the higher the electric energy consumption of the hp.
- for a first approach a suggest a constant target temperature for cooling set to target_Tc = 5.

### Electric power usage by hp and heater

In [ ]:
# Calculate the electric energy consumption of the hp for cooling
b_snip['electric_load_hp_kW'] = b_snip['cooling_load_kWh'] / b_snip['COP_c'] # add a column with the electric consumption [kW]
# Calculate the electric energy consumption of the heater for DHW heating
eta_heater = 0.99 # efficiency of the electric heater for DHW, usually close to 1
b_snip['electric_load_heater_kW'] = b_snip['dhw_heating_kWh']/eta_heater # add a column with the electric consumption of heater [kW]

In [ ]:
# electric power consumption of hp for cooling
plt.figure(figsize=(20,5))
sns.lineplot(data=b_snip,x='datetime',y='electric_load_hp_kW')
plt.title('Electric load of hp cooling');

In [ ]:
# electric power consumption of heater for DHW
plt.figure(figsize=(20,5))
sns.lineplot(data=b_snip,x='datetime',y='electric_load_heater_kW')
plt.title('Electric load of heater for DHW');


### Final plot with all electric powers
- with outdoor_temp as a reference
- with cooling_load_hp_kW as a reference

In [ ]:
# plotting power plots
# choose columns to plot
df_plot = b_snip.loc[:,('timestamp','outdoor_temp','cooling_load_kWh','electric_load_hp_kW','electric_load_heater_kW','solar_generation_kW','equipment_electric_power_kWh')]
# melt dataframe for plotting
df_plot = pd.melt(df_plot, 'timestamp', var_name='Load', value_name='Value') 
plt.figure(figsize=(20,5))
sns.lineplot(data=df_plot,x='timestamp',y='Value',hue='Load')
plt.title('Power of building over time')
plt.ylabel('Power [kW] / Temperature [°C]')
plt.grid('major')

In [ ]:
# Power residual
b_snip['power_residual'] = b_snip['solar_generation_kW'] \
                                - b_snip['electric_load_hp_kW'] \
                                - b_snip['electric_load_heater_kW'] \
                                - b_snip['equipment_electric_power_kWh']
plt.figure(figsize=(20,5))
sns.lineplot(data=b_snip,x='timestamp',y='power_residual')
plt.title('Power residual over time')
plt.ylabel('Power [kW]')
plt.grid('major')
